In [1]:
table = open("../data/paperTable.tsv","r")
entries = []
for line in table:
    entries.append(line.split('\t'))
table.close()

In [2]:
abstracts = []
titleofabstract = []
yearofabstract = []
for articles in entries:
    titleofabstract.append(articles[0])
    abstracts.append(articles[1]+articles[2]+articles[3])
    yearofabstract.append(articles[4][:-1])

In [3]:
# Create a set of frequent words
stopfile = open("../data/stopwords.txt","r")
stopwords = stopfile.read().splitlines()
stopwords.append("\xc2\xa9") #This is the copyright symbol, this shows up in every abstract and should not be apart of the corpus
stopwords.extend(["\u2019","\u03bc"])

stoplist = set(stopwords)
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in abstracts]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
processed_corpus = [[token for token in text if frequency[token] > 5] for text in texts]
# processed_corpus

In [4]:
from gensim import corpora
import os, tempfile
TEMP_FOLDER = tempfile.gettempdir()

dictionary = corpora.Dictionary(processed_corpus)
dictionary.save(os.path.join(TEMP_FOLDER,'words.dict'))
print(dictionary)

Dictionary(4611 unique tokens: [u'limited', u'clean-up', u'similarity', u'believed', u'proteins,']...)


In [5]:
new_doc = "limited bees important"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

corpus = [dictionary.doc2bow(text) for text in processed_corpus]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'words.dict'), corpus)
for c in corpus:
    print(c)

[(13, 1), (43, 1), (806, 1)]
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 5), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 3), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 2), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1)]
[(9, 2), (13, 1), (39, 1), (41, 1), (59, 1), (81, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 

In [16]:
from gensim.models import TfidfModel
from gensim import models

tfidf = TfidfModel(corpus,id2word=dictionary) 
lsi = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=25)

In [22]:
for i in xrange(0,24):
    print lsi.print_topics(8)
    print "\n\n"

[(0, u'-0.572*"[no" + -0.564*"abstract" + -0.552*"available]" + -0.057*"available]apis" + -0.035*"nosema" + -0.032*"varroa" + -0.031*"virus" + -0.025*"mellifera" + -0.025*"colony" + -0.024*"pollen"'), (1, u'-0.135*"varroa" + -0.131*"virus" + 0.127*"available]" + 0.121*"[no" + 0.119*"abstract" + -0.119*"pollen" + -0.115*"honey" + -0.114*"nosema" + -0.106*"colony" + -0.103*"pesticide"'), (2, u'0.296*"virus" + 0.193*"virus," + -0.177*"pesticide" + 0.166*"nosema" + 0.161*"varroa" + -0.157*"pollen" + 0.119*"paralysis" + -0.113*"pesticides" + 0.112*"mite" + -0.109*"mass"'), (3, u'0.236*"virus" + 0.220*"mass" + 0.201*"spectrometry," + 0.201*"chromatography," + 0.163*"liquid" + 0.148*"virus," + 0.142*"pesticide" + 0.131*"samples" + 0.128*"method" + 0.117*"tandem"'), (4, u'0.251*"varroa" + 0.230*"mite" + -0.229*"gene" + -0.176*"expression" + 0.169*"infestation" + 0.144*"colonies" + 0.139*"mites" + 0.131*"destructor" + -0.130*"immune" + -0.129*"rna"'), (5, u'-0.352*"virus" + 0.323*"nosema" + 0.2